## Imports and Data Initialization

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/ground_truth.csv", parse_dates=["date"])
# Need to find a way to remove the additional date 1900-01-01 while keeping datetime format. 
df["hours"]= pd.to_datetime(df["hours"], format = '%H:%M%S')
df.head()

,id_user,date,hours,id_item,price,qty
0,17850,2010-12-01,1900-01-01 08:02:06,85123A,2.55,6
1,17850,2010-12-01,1900-01-01 08:02:06,71053,3.39,6
2,17850,2010-12-01,1900-01-01 08:02:06,84406B,2.75,8
3,17850,2010-12-01,1900-01-01 08:02:06,84029G,3.39,6
4,17850,2010-12-01,1900-01-01 08:02:06,84029E,3.39,6


In [3]:
df["id_item"]=df["id_item"].astype("category")
df["date"]=df["date"].astype("category")
categorical = set ({'date','id_item'})

In [4]:
# Initial MU : 14.1+ MB
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307054 entries, 0 to 307053
Data columns (total 6 columns):
id_user    307054 non-null int64
date       307054 non-null category
hours      307054 non-null datetime64[ns]
id_item    307054 non-null category
price      307054 non-null float64
qty        307054 non-null int64
dtypes: category(2), datetime64[ns](1), float64(1), int64(2)
memory usage: 10.7 MB


In [5]:
#Quasi identification sur base de date et d'heure
df[(df["date"]=="2010-12-01") & (df["hours"]=="1900-01-01 08:02:06")]

,id_user,date,hours,id_item,price,qty
0,17850,2010-12-01,1900-01-01 08:02:06,85123A,2.55,6
1,17850,2010-12-01,1900-01-01 08:02:06,71053,3.39,6
2,17850,2010-12-01,1900-01-01 08:02:06,84406B,2.75,8
3,17850,2010-12-01,1900-01-01 08:02:06,84029G,3.39,6
4,17850,2010-12-01,1900-01-01 08:02:06,84029E,3.39,6
5,17850,2010-12-01,1900-01-01 08:02:06,22752,7.65,2
6,17850,2010-12-01,1900-01-01 08:02:06,21730,4.25,6


In [6]:
# Number of unique users

df["id_user"].nunique()

4034

In [7]:
# The quantity bought by user 17850 is very often 6 (215/297=72%)

#print(df[df["id_user"]==12680].head(1000).to_string())
df[df["id_user"]==17850]["qty"].value_counts()

6     215
4      31
2      24
12     15
8      11
3       1
Name: qty, dtype: int64

In [8]:
# The item 84406B is most bought by user 17850 (15/176 = 8% vs 0.9% mean per other user so more than x8 times )

df[df["id_item"]=="84406B"]["id_user"].value_counts()
#df[df["id_item"]=="84406B"]["id_user"].value_counts().mean()

17850    15
17858     6
17191     5
15708     4
12775     4
         ..
12956     1
13474     1
14499     1
17580     1
14687     1
Name: id_user, Length: 106, dtype: int64

In [9]:
# The user 12688 is the one who most bought at the date 2011-08-18

df[df["date"]=="2011-08-18"]["id_user"].value_counts()

12688    171
15472     76
15867     54
16813     38
16767     34
14221     32
13381     31
18225     31
14565     31
17750     31
13048     29
16743     28
16187     28
14189     27
14064     26
12729     26
15301     25
17736     23
15615     22
18272     22
12839     20
16261     20
17720     20
12680     20
17576     18
17243     18
14132     18
15189     18
13319     17
16945     16
17045     16
13273     16
15743     16
15024     15
13617     15
15152     14
15144     14
13113     14
15232     13
13014     13
17001     13
15125     12
12962     11
14741      9
15505      7
16928      7
16582      6
13027      5
16626      4
14051      4
13576      3
13784      3
17386      2
17742      2
15400      1
15797      1
14305      1
Name: id_user, dtype: int64

In [10]:
{df["qty"].min(),df["qty"].max()}

{1, 4800}

# Implementation of k-anonymity

In [11]:
def get_spans(df,partition, scale = None):
    spans={}
    for column in df.columns:
        if column in categorical:
            span=len(df[column][partition].unique())
        else:
            span=df[column][partition].max()-df[column][partition].min()
        if scale is not None:
            span = span/scale[column]
        spans[column]=span
    return spans

In [12]:
full_spans = get_spans(df,df.index)
full_spans

{'id_user': 5940,
 'date': 305,
 'hours': Timedelta('0 days 13:59:06'),
 'id_item': 3612,
 'price': 8142.749,
 'qty': 4799}

In [13]:
def split(df, partition, column):
    
    """""
    :param     df: The dataframe from which we want to split a partition
    :param     partition: The range of indexes (rows) we want to select from df 
    :param     column: The column to select from df, and operate the split with
    
    Example : 
    
    index         age(numerical)   gender(categorical)
      0            19              female
      1            25              female
      2            14              male
      3            51              female

      
      If column = age 
      Median = (S[2]+S[1])/2 = (14+25)/2 = 19.5 ; Sorted age : {14,19,25,51} = {S[2],S[0],S[1],S[3]}
      return (Range{2,0},Range{1,3})
      
      If column=gender (It is precisely because of even cardinals that we can't calculate medians for non numerical)
      values = {gender,female}
      return (Range{2},Range{0,1,3}) 
      
      As shown above, cases where the partition is split on a non numerical column, can result in a weird situation
      where left and right partition don't really have the same number of values.
      And this is only a showcase situation, it is virtually possible to have 0 value at left, and all values at right
      (which also happens when the quasi identifier has only one value in the partition : card(values)=1)
      or 75% at left and 25% at right etc... 
      
      So really for categorical values, the notion of median (which is supposed to split the set in 50%) doesn't make
      any sens here.

    """""
    
    
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                                            dfp = df[column][partition]
                                            
                          df[column] is a set (like an array one dimension, index : value)
                          df[column][partition] is a subset of df[column] (selection of rows)
                          It is also treated as a set
                          
                            Example: 
                            
                            df[age] = 0 19 = df[df.index]
                                      1 25
                                      2 14
                                      3 51
                            
                          df[column][0] = 19 
                          df[column][[0,1]]= 19
                                             25
                          [0,1] can be viewed as the range of indexes from 0 to 1 included
                          df.index is the range of all indexes from 0 to df.len-1
                          partition is nothing more than the set of indexes of a given number of rows
                            
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
   
     
    
    dfp = df[column][partition]
    
    if ((column in categorical) | (column == "date") | (column == "hours")):
        values = dfp.unique()
        lv=set(values[:len(values)//2])
        rv=set(values[len(values)//2:])
        return dfp.index[dfp.isin(lv)],dfp.index[dfp.isin(rv)]
    else:
        median=dfp.median()
        #print(dfp, median)
        dfl=dfp.index[dfp<median]
        dfr=dfp.index[dfp>=median]
        #print(df["qty"][dfl])
        return(dfl,dfr)

In [14]:
39/2

19.5

In [15]:
dfl,dfr = split(df,df.index,"id_item")
dfl,dfr

(Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                  8,      9,
             ...
             307040, 307041, 307042, 307043, 307044, 307045, 307046, 307047,
             307049, 307050],
            dtype='int64', length=217230),
 Int64Index([  9985,  10015,  10062,  10074,  10078,  10110,  10137,  10144,
              10169,  10173,
             ...
             307019, 307023, 307030, 307032, 307033, 307036, 307048, 307051,
             307052, 307053],
            dtype='int64', length=89824))

In [16]:
# If partition has less members than k then it's not a valid one, so we can't further divide it
def is_k_anonymous (df, partition, sensitive_column, k=5000):
    if len(partition)<k:
        return False
    return True

In [17]:
def partition_dataset(df, feature_columns,sensitive_column, scale, is_valid):  
    """""
    :param     df: The dataframe to partition
    :param     feature_columns: The Quasi-Identifier columns 
    :param     sensitive_column : The Sensitive Data we wish to protect
    :param     scale : original full_spans of df before the first split
    
    """""
    finished_partitions=[]
    
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                                            [df.index]
                                            
                    df.index is a range of indexes, it can be seen as an array of indexes while it's not 
                    technically an array.
                    [df.index] in between brackets simply means you're initiliazing a list of index ranges, 
                    you can see it as an array of arrays.
                            
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    partitions=[df.index]
    
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                                            while partitions:
                                            
                    The main loop here works like a chef, think of partitions like a cucumber, 
                    each time we're going through the while, chef cuts it in two equal parts (more or less **)
                                    (i1) <----> => (i2) <--><--> =>  (i3) <-><-><-><->
                                        PS: Sorry for the poor drawing skills
                        ** Refer to the split method to understand this remarque.
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    while partitions:
        
        # Pop the oldest partition from the list of partitions, so that we can split it further (or at least try).
        # Remember partition is the same type as df.index so it's a range of indexes (an "array" of indexes)
        
        partition = partitions.pop(0)
        
        # Update the spans (number of unique values for each quasi-identifier column) for this iteration
        
        spans = get_spans(df[feature_columns],partition, scale)
        
        """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                            for column, span in sorted(spans.items(), key = lambda x:-x[1]):
                    
                    for {column,span} in {column1 : numberOfUniqueItemInColumn_1    
                                          column2 : numberOfUniqueItemInColumn_2   
                                          ....
                                          columnN : numberOfUniqueItemInColumn_N   
                                          } 
                    where numberOfUniqueItemInColumn_i is a drecreasing sequence (suite décroissante) 
                    
                    Here we loop through the quasi identifiers (since span stores the quasi identifier column
                    names and values) ordered in decreasing order of multiplicity (number of # values).
                    
                    
                    Details :
                    
                    spans.items() returns a hashmap of spans column names as keys, and span column values as values
                    
                    key = lambda x:-x[1] tells to sorted(), hey I want you to sort this hashmap in decreasing order 
                    of the elements that are in [1] so in decreasing values in this case. 
                    (key = labmda x:x[0] would've returned a sorted hashmap in increasing order of indexes which 
                    is the default)
             
                    
        """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    
        for column, span in sorted(spans.items(), key = lambda x:-x[1]):
            
            # lp and rp are like partition : they are a range of indexes (an "array" of indexes)
            # lp is the ranges of indexes for whome df[column][lp] < median
            # rp is the ranges of indexes for whome df[column][rp] >= median

            lp, rp = split(df,partition,column)
            
            """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                    if not is_valid(df,lp,sensitive_column) or not is_valid(df,rp,sensitive_column):
                                            
                    As long as one of both split partitions is still valid (len(partition)>k) we need
                    to break from the for loop (don't even look for next quasi identifier) and add lp and rp 
                    to the list of partitions we want to cut even more.
                    
                    Otherwise we enter the if, and execute continue : 
                    What continue does, is forget the rest of the loop and skip to the next iteration,
                    so select the next quasi identifier to divide the partition.
                    
                    If we tried with all quasi identifiers, and none worked, then our partition is done (as
                    small as possible) and when we execute continue it will have no more quasi to explore. 
                    So executes the else and append the finished partition to the set of finished_partitions.
        
                            
            """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
            if not is_valid(df,lp,sensitive_column) or not is_valid(df,rp,sensitive_column):
                continue
            partitions.extend((lp,rp))
            break
        else:
            finished_partitions.append(partition)
    return finished_partitions

In [18]:
feature_columns=["date", "qty"]
sensitive_column="price"
finished_partitions=partition_dataset(df,feature_columns,sensitive_column, full_spans, is_k_anonymous)

In [19]:
len(finished_partitions)
#df["date"][finished_partitions[0]]
#finished_partitions[0]
#finished_partitions

44

In [20]:
len(df)/305
df["qty"][finished_partitions[5]].value_counts()

1    2491
2    2330
4    1393
3    1060
5     332
Name: qty, dtype: int64

In [21]:
"""
Various handlers for .agg() method to call on each column of df.loc[partition] depending on the column type

:param   series: Column Set (from df.loc[partition]) to be processed 
"""

def agg_categorical_column(series):
    return[','.join(set(series))]

def agg_numerical_column(series):
    return[series.mean()]

def agg_date_column(series):
    # remove the hh:mm:ss
    #series=series.dt.date
    if(isinstance(series,pd.Series)):
        print(series[series.idxmin], series[series.idxmax])
        return[[series[series.idxmin], series[series.idxmax]]]
    #return [[series.max(),series.min()]]
    return ["something"]


In [22]:
def build_anonymized_dataset(df,partitions,feature_columns,sensitive_column,max_partitions=None):
    aggregations = {}
    for column in feature_columns:
        if column in categorical:
            if column == "date":
                aggregations[column]=agg_date_column
            else:
                aggregations[column]=agg_categorical_column
            
        else: 
            aggregations[column]=agg_numerical_column
            
    rows=[]
    for i, partition in enumerate(partitions):
        if i%100==1:
            print("Finished {} partitions ! ".format(i) )
        if max_partitions is not None and i > max_partitions:
            break
        #df.agg({column : method_to_apply})
        #df.agg({numerical_column : agg_numerical_column
        #         categorical_column : aggww_categorical_column
        #        })
        # Result 1 line multiple column, each cell=aggreg result
        
        #print(aggregations)
        #print(df.loc[partition].agg(aggregations,squeeze=False))
        #print(type(df.loc[partition]["date"]))
        grouped_columns=df.loc[partition].agg(aggregations,squeeze=False)
        # Count spans of sensitive column in a partition
        sensitive_counts = df.loc[partition].groupby(sensitive_column).agg({
            sensitive_column : 'count'
        })
        values = grouped_columns.iloc[0].to_dict()
        #print(values)
        for sensitive_value, count in sensitive_counts[sensitive_column].items():
            if count==0:
                continue
            values.update({
                sensitive_column : sensitive_value,
                'count' : count,
            })
            rows.append(values.copy())
    return pd.DataFrame(rows)


In [23]:
dfn=build_anonymized_dataset(df,finished_partitions,feature_columns,sensitive_column)

2010-12-01 00:00:00 2010-12-22 00:00:00
Finished 1 partitions ! 
2010-12-23 00:00:00 2011-01-24 00:00:00
2011-01-25 00:00:00 2011-02-15 00:00:00
2011-02-16 00:00:00 2011-03-09 00:00:00
2011-03-10 00:00:00 2011-03-31 00:00:00
2011-04-01 00:00:00 2011-04-26 00:00:00
2011-04-27 00:00:00 2011-05-20 00:00:00
2011-05-22 00:00:00 2011-06-13 00:00:00
2010-12-01 00:00:00 2011-01-24 00:00:00
2011-01-25 00:00:00 2011-03-09 00:00:00
2011-03-10 00:00:00 2011-04-26 00:00:00
2011-04-27 00:00:00 2011-06-13 00:00:00
2011-01-25 00:00:00 2011-03-09 00:00:00
2011-06-14 00:00:00 2011-07-05 00:00:00
2011-07-06 00:00:00 2011-07-27 00:00:00
2011-07-28 00:00:00 2011-08-18 00:00:00
2011-08-19 00:00:00 2011-09-11 00:00:00
2011-09-12 00:00:00 2011-10-03 00:00:00
2011-06-14 00:00:00 2011-07-27 00:00:00
2011-07-28 00:00:00 2011-09-11 00:00:00
2011-06-14 00:00:00 2011-09-11 00:00:00
2011-06-14 00:00:00 2011-12-09 00:00:00
2010-12-01 00:00:00 2011-01-24 00:00:00
2010-12-01 00:00:00 2011-01-24 00:00:00
2011-03-10 00:0

In [24]:
sectors=dfn.groupby("date")
#dfn["date"].value_counts().head(60)

In [25]:
dfn.to_csv("k_anon:date+qty:price.csv")

In [26]:
dfn

,date,qty,price,count
0,"[2010-12-01 00:00:00, 2010-12-22 00:00:00]",2.134386,0.14,2
1,"[2010-12-01 00:00:00, 2010-12-22 00:00:00]",2.134386,0.16,1
2,"[2010-12-01 00:00:00, 2010-12-22 00:00:00]",2.134386,0.18,1
3,"[2010-12-01 00:00:00, 2010-12-22 00:00:00]",2.134386,0.19,4
4,"[2010-12-01 00:00:00, 2010-12-22 00:00:00]",2.134386,0.21,12
...,...,...,...,...
3765,"[2011-06-14 00:00:00, 2011-12-09 00:00:00]",114.259662,10.40,1
3766,"[2011-06-14 00:00:00, 2011-12-09 00:00:00]",114.259662,10.95,21
3767,"[2011-06-14 00:00:00, 2011-12-09 00:00:00]",114.259662,12.48,1
3768,"[2011-06-14 00:00:00, 2011-12-09 00:00:00]",114.259662,13.35,1


In [27]:
df["date"]

0        2010-12-01
1        2010-12-01
2        2010-12-01
3        2010-12-01
4        2010-12-01
            ...    
307049   2011-12-09
307050   2011-12-09
307051   2011-12-09
307052   2011-12-09
307053   2011-12-09
Name: date, Length: 307054, dtype: category
Categories (305, datetime64[ns]): [2010-12-01, 2010-12-02, 2010-12-03, 2010-12-05, ..., 2011-12-06, 2011-12-07, 2011-12-08, 2011-12-09]

In [28]:
#df["date"].min()-df["date"].max()

In [29]:
df["date"]=pd.DatetimeIndex(df["date"]).month

In [30]:
df[df["date"]=="2010-12-01"]["id_user"].value_counts()

/Users/hussein/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Series([], Name: id_user, dtype: int64)

In [31]:
df["id_item"].nunique()

3612